In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Data Cleaning

<img src="docu/pictures/data-cleaning.webp" style="width: 60%; align: center;" />

## Loading the Data

In [2]:
from os import listdir


def load_bios(path: str) -> list:
    bios = list()
    for f in listdir(path):
         if f.endswith('-bio.txt'):
                with open(f'{path}/{f}', 'r') as txt:
                    bio = txt.read()
                bios.append(bio)
    return bios


bios = load_bios('../opendata-music-artists-with-bios/bios/')[:100]
bios[0]

'Rock Metal from northwestern Spain. The band is still unsigned and  with three works on their backs. Their last release it\'s an acoustic experiment with three electric bonus tracks and it will be release on June 2006.\n\nCheck it out\nhttp://www.orfos.tk\nhttp://www.myspace.com/orfos <a href="https://www.last.fm/music/Orfos">Read more on Last.fm</a>. User-contributed text is available under the Creative Commons By-SA License; additional terms may apply.'

## Removing Ambiguous Data

In [3]:
def is_bio(text: str) -> bool:
    if "at least" in text[:200]:
        return False
    elif text.lower().startswith('there are'):
        return False
    elif text.lower().startswith('there is'):
        return False
    return True


real_bios = [b for b in bios if is_bio(b)]
false_bios = list(set(bios).difference(set(real_bios)))
print(f"{len(false_bios)}/{len(bios)} filtered out")

5/100 filtered out


## Removing HTML Code, Control Chars, URLs

In [4]:
import re


def remove_html_tags(text: str) -> str:
    cleaner = re.compile('<.*?>')
    return re.sub(cleaner, '', text)

bios_wo_html = [remove_html_tags(b) for b in real_bios]
bios_wo_html[0]

"Rock Metal from northwestern Spain. The band is still unsigned and  with three works on their backs. Their last release it's an acoustic experiment with three electric bonus tracks and it will be release on June 2006.\n\nCheck it out\nhttp://www.orfos.tk\nhttp://www.myspace.com/orfos Read more on Last.fm. User-contributed text is available under the Creative Commons By-SA License; additional terms may apply."

In [5]:
def remove_control_chars(text: str) -> str:
    text = text.replace('\n',' ')
    text = text.replace('\t',' ')
    text = text.replace('\r',' ')
    return text


bios_wo_control_chars = [remove_control_chars(b) for b in bios_wo_html]
bios_wo_control_chars[0]

"Rock Metal from northwestern Spain. The band is still unsigned and  with three works on their backs. Their last release it's an acoustic experiment with three electric bonus tracks and it will be release on June 2006.  Check it out http://www.orfos.tk http://www.myspace.com/orfos Read more on Last.fm. User-contributed text is available under the Creative Commons By-SA License; additional terms may apply."

In [6]:
def remove_urls(text: str) -> str:
    return re.sub(r'http\S+', '', text)

bios_wo_urls = [remove_urls(b) for b in bios_wo_control_chars]
bios_wo_urls[0]

"Rock Metal from northwestern Spain. The band is still unsigned and  with three works on their backs. Their last release it's an acoustic experiment with three electric bonus tracks and it will be release on June 2006.  Check it out   Read more on Last.fm. User-contributed text is available under the Creative Commons By-SA License; additional terms may apply."

In [7]:
bios_wo_lastfm_text = [b.split('Read more on Last.fm')[0] for b in bios_wo_urls]
bios_wo_lastfm_text[0]

"Rock Metal from northwestern Spain. The band is still unsigned and  with three works on their backs. Their last release it's an acoustic experiment with three electric bonus tracks and it will be release on June 2006.  Check it out   "

## Converting Words to Their Root Form

In [8]:
import spacy

In [9]:
# !spacy download en_core_web_sm

In [10]:
def lemmatize(text: str, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']) -> str:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    doc = nlp(text)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postags:
            new_text.append(token.lemma_)
    return ' '.join(new_text)

from joblib import Parallel, delayed
bios_lemmatized = Parallel(n_jobs=8, verbose=5, backend="loky")(map(delayed(lemmatize), bios_wo_lastfm_text))


# bios_lemmatized = [lemmatize(b) for b in bios_wo_lastfm_text]
bios_lemmatized[0]

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/michael/workspace/topic-modelling-workshop/venv/lib/python3.10/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
/Users/michael/workspace/topic-modelling-workshop/venv/lib/python3.10/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(
/Users/michael/workspace/topic-modelling-workshop/venv/lib/python3.10/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipe

'northwestern band still unsigned work back last release acoustic experiment electric bonus track release check'

## Creating List of Words

In [11]:
import gensim


def gen_words(text: str) -> list:
    return gensim.utils.simple_preprocess(text, deacc=True)


data_words = [gen_words(b) for b in bios_lemmatized]
data_words[0]

['northwestern',
 'band',
 'still',
 'unsigned',
 'work',
 'back',
 'last',
 'release',
 'acoustic',
 'experiment',
 'electric',
 'bonus',
 'track',
 'release',
 'check']

# Create LDA Model

<img src="docu/pictures/creating-the-model.jpg" style="width: 60%; align: center;" />

In [12]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import LdaModel


id2word = corpora.Dictionary(data_words)
corpus = [id2word.doc2bow(text) for text in data_words]
word = id2word[[0][:1][0]]
print (corpus[0][0:20])
print (word)

lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=15,
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha="auto")

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1)]
acoustic


# Visualization

<img src="docu/pictures/visualization.jpg" style="width: 60%; align: center;" />

In [13]:
import pyLDAvis
import pyLDAvis.gensim_models


pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/Users/michael/workspace/topic-modelling-workshop/venv/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/michael/workspace/topic-modelling-workshop/venv/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/michael/workspace/topic-modelling-workshop/venv/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/michael/workspace/topic-modelling-workshop/venv/lib/python3.10/site-packages/past/builtins/mi

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.088777  0.186623       1        1  41.245577
9     -0.023537  0.210026       2        1  16.292691
2     -0.237954  0.002986       3        1   8.902998
10     0.112141  0.187654       4        1   8.599319
0      0.233358  0.021074       5        1   7.370895
8     -0.113327  0.064319       6        1   3.760193
7      0.107193 -0.162653       7        1   3.030950
4      0.097407  0.079564       8        1   2.981488
13    -0.076439 -0.215212       9        1   2.384303
5     -0.112018 -0.080680      10        1   1.947156
6      0.041800 -0.004623      11        1   1.219878
12     0.079297 -0.066929      12        1   1.118369
11    -0.025024 -0.069695      13        1   0.523179
1      0.007999 -0.094992      14        1   0.487515
3     -0.002120 -0.057463      15        1   0.135490, topic_info=        Term        Freq       Total Category  logprob  loglift
2       band  176.000000  176.000000  Default  30.0000  30.0000
195    album  130.000000  130.000000  Default  29.0000  29.0000
9    release  109.000000  109.000000  Default  28.0000  28.0000
46     music   96.000000   96.000000  Default  27.0000  27.0000
368   guitar   32.000000   32.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
79     first    0.004903   43.699560  Topic15  -7.6793  -2.4912
49       new    0.004847   47.288235  Topic15  -7.6909  -2.5817
69      year    0.004815   44.064015  Topic15  -7.6975  -2.5177
368   guitar    0.004794   32.891579  Topic15  -7.7017  -2.2295
361   follow    0.004774   26.881502  Topic15  -7.7061  -2.0321

[964 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1409      4  0.728091     abuse
719       7  0.854891    accent
1269      8  0.854848   account
0         1  0.843579  acoustic
0         6  0.105447  acoustic
...     ...       ...       ...
509       1  0.439439       yet
509       2  0.146480       yet
509       3  0.292959       yet
2206      3  0.723539    zombie
707       9  0.895331  владимир

[1364 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 10, 3, 11, 1, 9, 8, 5, 14, 6, 7, 13, 12, 2, 4])